In [ ]:
import numpy as np
import os
import pandas as pd
import nbimporter
from pred import preprocess_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import statsmodels.api as sm

In [36]:
df = pd.read_csv('../cleaned_data/merged_data.csv')

In [37]:
X = df[['County', 'State', 'Median income', 'Mean income', 'Sex', 'ethnicity']]
y = df['Heart Disease Mortality']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [39]:
X_train, X_test, y_train, y_test, preprocessor = preprocess_data(df)

In [40]:
rf_regressor_optimized = Pipeline([
    ('preprocessor', preprocessor), ('polyfeatures', PolynomialFeatures()), 
    ('regressor', RandomForestRegressor(random_state = 42, n_estimators = 100, max_depth = None, max_features = None, min_samples_split=3, min_samples_leaf=3))
    ])


In [41]:
rf_regressor_optimized.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
rf_model = rf_regressor_optimized.named_steps['regressor']

# Get feature importance
feature_importance = rf_model.feature_importances_

In [ ]:
rf_regressor_optimized.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['County', 'State',
                                                   'Median income',
                                                   'Mean income', 'Sex',
                                                   'ethnicity'])])),
                ('regressor',
                 RandomForestRegressor(max_features=None, min_samples_leaf=3,
                                       min_samples_split=3, random_state=42))])

In [ ]:
y_hat_test = rf_regressor_optimized.predict(X_test)

In [ ]:
test_mse = mean_squared_error(y_test, y_hat_test)
r_squared = r2_score(y_test, y_hat_test)
print(test_mse)
print(r_squared)

6902.595358688841
0.1975307933299162


In [ ]:
rf_regressor_optimized.named_steps['regressor'].feature_importances_

In [ ]:
sm.qqplot(std_residuals, line="45")
plt.show()